In [1]:
import sqlalchemy
sqlalchemy.create_engine('sqlite:///foo.db')

Engine(sqlite:///foo.db)

In [2]:
# ! pip install ipython-sql

In [3]:
%load_ext sql

In [4]:
%sql sqlite:///foo.db

In [5]:
%%capture 
%load_ext sql 
%sql sqlite:///foo.db

In [6]:
%%sql

create table winsales(
salesid int,
dateid date,
sellerid int,
buyerid char(10),
qty int,
qty_shipped int);

insert into winsales values
(10003, '8/2/2003', 3, 'b', 8, 10),
(10001, '12/24/2003', 1, 'c', 10, 10),
(10005, '12/24/2003', 1, 'a', 30, null),	
(40001, '1/9/2004', 4, 'a', 40, null),	
(10006, '1/18/2004', 1, 'c', 10, null),	
(20001, '2/12/2004', 2, 'b', 20, 20),
(40005, '2/12/2004', 4, 'a', 10, 10),
(20002, '2/16/2004', 2, 'c', 20, 20),
(30003, '4/18/2004', 3, 'b', 15, null),
(30004, '4/18/2004', 3, 'b', 20, null),	
(30007, '9/7/2004', 3, 'c', 30, null);	

 * sqlite:///foo.db
(sqlite3.OperationalError) table winsales already exists
[SQL: create table winsales(
salesid int,
dateid date,
sellerid int,
buyerid char(10),
qty int,
qty_shipped int);]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [7]:
%%sql

SELECT * FROM winsales

 * sqlite:///foo.db
Done.


salesid,dateid,sellerid,buyerid,qty,qty_shipped
10003,8/2/2003,3,b,8,10
10001,12/24/2003,1,c,10,10
10005,12/24/2003,1,a,30,None
40001,1/9/2004,4,a,40,None
10006,1/18/2004,1,c,10,None
20001,2/12/2004,2,b,20,20
40005,2/12/2004,4,a,10,10
20002,2/16/2004,2,c,20,20
30003,4/18/2004,3,b,15,None
30004,4/18/2004,3,b,20,None


- Ко всем транзакциям пользователя вывести дату первой покупки

- К каждой транзакции добавить дату предыдущей транзакции пользователя

- Показать сумму покупок пользователя нарастающим итогом

- Присвоить всем транзакциям пользователя / продавца / отделения порядковый номер

In [8]:
%%sql

SELECT *, 
min (dateid) over (partition by buyerid) as first_prchs_id,
LAG(dateid) over (partition by buyerid order by dateid) as prev_prchs_date,
SUM (qty) over (partition by buyerid order by dateid) as buyer_qty_cum_sum,
ROW_NUMBER() over (partition by buyerid order by dateid) as buyer_prchs_num,
ROW_NUMBER() over (partition by sellerid order by dateid) as seller_purchs_num
FROM winsales
ORDER BY buyerid, dateid

 * sqlite:///foo.db
Done.


salesid,dateid,sellerid,buyerid,qty,qty_shipped,first_prchs_id,prev_prchs_date,buyer_qty_cum_sum,buyer_prchs_num,seller_purchs_num
40001,1/9/2004,4,a,40,None,1/9/2004,None,40,1,1
10005,12/24/2003,1,a,30,None,1/9/2004,1/9/2004,70,2,3
40005,2/12/2004,4,a,10,10,1/9/2004,12/24/2003,80,3,2
20001,2/12/2004,2,b,20,20,2/12/2004,None,20,1,1
30003,4/18/2004,3,b,15,None,2/12/2004,2/12/2004,55,2,1
30004,4/18/2004,3,b,20,None,2/12/2004,4/18/2004,55,3,2
10003,8/2/2003,3,b,8,10,2/12/2004,4/18/2004,63,4,3
10006,1/18/2004,1,c,10,None,1/18/2004,None,10,1,1
10001,12/24/2003,1,c,10,10,1/18/2004,1/18/2004,20,2,2
20002,2/16/2004,2,c,20,20,1/18/2004,12/24/2003,40,3,2
